In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"ouput": "How are you?"})

memory.load_memory_variables({})

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

chat = ChatOpenAI(
    temperature=1,
    model_name="gpt-5-nano",
    streaming=True,
    tiktoken_model_name="gpt-3.5-turbo",
    )

examples = [
    {
        "movie": "The Shawshank Redmption",
        "answer": "🔨⛓️🕊️"
    },
     {
        "movie": "The Dark Knight",
        "answer": "🦇🃏💥"
    },
     {
        "movie": "Inception",
        "answer": "💤🌀🎯"
    },
]

example_template  = ChatPromptTemplate.from_messages([
    ("human", "Summarize this {movie} with three emoticons"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_template,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a master at summarizing emojis"),
    example_prompt,
    ("human", "Summarize the {movie} with emoticons")
])

chain = final_prompt | chat

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=100,
    memory_key="chat_history"
)

movie_chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=final_prompt,
)

chat_template = """
    You are a helpful AI talking a human.

    {chat_history}
    human:{question}
    you:
"""

conversation_chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=PromptTemplate.from_template(chat_template),
)



In [21]:
movie_chain.predict(movie="Parasite")
movie_chain.predict(movie="The Matrix")

'🕶️💊🤖'

In [22]:
conversation_chain.predict(
    question="What movies did I ask you to summarize?"
)

'Parasite and The Matrix. Would you like me to summarize both, or just one?'